# Indeed JobScrape

This notebook is created to demonstrate that web scraping can be perform in python using beatifulsoup library and html. 
In 1 minute, this bot is able to scrape through close to 1000 records of job listings from Indeed.com.

⚠️**WARNING* ⚠️ *Run this program at your own risk. Do not run this web scraping program repeatedly** as it can be result in you getting ban by Indeed.com.



In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

def get_url(position, location):
    """
    Generate a url from position and location
    """
    template = 'https://sg.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url
    
def get_record(card):
    """
    Extract job data from  a single record
    """
    atag = card.h2.a
    atag1 = card.h2.a.span
    job_title = atag1.get('title')
    job_url = 'https://sg.indeed.com'+ atag.get('href')
    try:
        company = card.find('span','companyName').text.strip()
    except AttributeError:
        company = ''
    job_location = card.find('div','companyLocation').text.strip()
    job_summary = card.find('div','job-snippet').text.strip()
    post_date = card.find('span','date').text.strip()
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('div',{'class':'metadata salary-snippet-container'}).get_text()
    except AttributeError:
        job_salary = ''
    
    #Creates a tuple object
    record = (job_title, company, job_location, post_date, today, job_salary, job_summary, job_url)
    
    return record

def main(position, location):
    """
    Run the main program routine
    """
    records = []
    url = get_url(position, location)
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')
    
        for card in cards:
            record = get_record(card)
            records.append(record)

        try:
            url = 'https://sg.indeed.com'+ soup.find('a', {'aria-label': "Next"}).get('href')
        except AttributeError:
            break
            
    # save the job data
    with open('results.csv', 'w', newline = '', encoding = 'utf-8') as f: 
        writer = csv.writer(f)
        writer.writerow(['Job Title', 'Company', 'Location', 'Posting Date', 'Extract Date', 'Salary', 'Summary', 'JobUrl'])
        writer.writerows(records)

In [2]:
JobTitle = input("Please Provide the job title")

Please Provide the job title data analyst


In [3]:
%%time

# run the main program
main(JobTitle, 'Singapore')

Wall time: 51.1 s
